In [2]:
import torch
import torch.nn as nn

In [3]:
model = nn.Sequential()
model.add_module('conv1', nn.Conv2d(1, 32, kernel_size=3, padding='same'))
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(2))
model.add_module('conv2', nn.Conv2d(32, 64, kernel_size=3, padding='same'))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(2))
model.add_module('flatten', nn.Flatten())
model.add_module('dense1', nn.Linear(3136, 100))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(0.3))
model.add_module('dense2', nn.Linear(100, 10))

In [4]:
model.load_state_dict(torch.load('best_cnn_model.pt', weights_only=True))

<All keys matched successfully>

In [8]:
# 모델의 층 목록 - model.children()
layers = [layer for layer in model.children()]

In [9]:
model.conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)

In [ ]:
# model.named_children() : (층의 이름, 층 객체)

In [10]:
for name, layer in model.named_children():
    print("name:", name, "layer:", layer)

name: conv1 layer: Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
name: relu1 layer: ReLU()
name: pool1 layer: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
name: conv2 layer: Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
name: relu2 layer: ReLU()
name: pool2 layer: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
name: flatten layer: Flatten(start_dim=1, end_dim=-1)
name: dense1 layer: Linear(in_features=3136, out_features=100, bias=True)
name: relu3 layer: ReLU()
name: dropout layer: Dropout(p=0.3, inplace=False)
name: dense2 layer: Linear(in_features=100, out_features=10, bias=True)


In [ ]:
# 특성맵 시각화 - 첫번째 Conv 층 통과한 특성맵

# 패션 MNIST 데이터 
from torchvision.datasets import FashionMNIST

fm_train = FashionMNIST(root='.', train=True, download=True)
train_input = fm_train.data